In [1]:
using HTTP, JSON, PrettyTables, JLD, DotEnv, Distributions, LinearAlgebra, Dates, MySQL

In [2]:
cfg = DotEnv.config("../.env")
files_path = cfg["files_path"]
adjacency = load(files_path*"placement_rates.jld")["placement_rates"]
classification_properties = load(files_path*"classification_properties.jld")["classification_properties"]
refresh_mysql_db = true
classification_properties

Dict{String, Any} with 7 entries:
  "data_loaded"              => DateTime("2024-08-31T23:13:46.046")
  "row_labels"               => Any["TYPE 1 (20 insts)", "TYPE 2 (62 insts)", "…
  "unmatched_index"          => 10
  "number_of_academic_types" => 5
  "institution_counts"       => [20, 62, 179, 352, 568, 159, 255, 635, 443, 1, …
  "algorithm_run_id"         => 6
  "num_years"                => 24

# Likelihood Ratios

The objective is to calculate the probability of each individual university's profiles of hires and placements given the placement rates estimated for each of the tiers in the classification.  The probabilities are then compared across tiers by calculating the ratio of each of these probabilities to the probability they have given the tier in which the university was placed by the classification algorithm.  If a university is classified as tier 1, then the probabililty of its hires and placements should he bigher using the rates estimated for tier 1 than they are using the rates estimated for tier 2.

We'll actually do it for hires and placements separately as well to check how restrictive is the assumption that universities who are the best at placing students are also best at hiring them.


In [3]:
# tier hiring rates
# this is just for illustration - the hiring rate should be per year per university
# adjacency is the matrix returned by the classification algorithm 
phr = zeros(size(adjacency)[1], size(adjacency)[2]);
for i in 1:size(adjacency)[1], j in 1:size(adjacency)[2]
    phr[i,j] = 
    adjacency[i,j]/(classification_properties["num_years"]*classification_properties["institution_counts"][i])
end
phr

12×5 Matrix{Float64}:
  2.56667       0.766667     0.379167     0.102083     0.0
  0.665323      0.659946     0.21707      0.0819892    0.00537634
  0.249302      0.335196     0.238594     0.0363128    0.00768156
  0.0332623     0.0671165    0.0546875    0.0557528    0.00508996
  0.000586854   0.00344777   0.0037412    0.0032277    0.0118838
  0.151205      0.156971     0.076782     0.0183438    0.00524109
  0.129248      0.075        0.0413399    0.0125817    0.00457516
  0.00570866    0.0104331    0.0093832    0.00426509   0.00190289
  0.00329195    0.00686606   0.00714823   0.00470278   0.00507901
 17.0417       31.0833      39.5833      24.1667      18.375
  0.226496      0.192308     0.104701     0.0373932    0.00961538
  0.0147059     0.0235671    0.0267094    0.0112494    0.0043992

The third to last row above aggragates all the placements that ended up in intitution 574 (ocean and crow). 
Each of these placements was either a applicant who never left a trail to show where they were hired, or an applicant
who got a job in an institution that was not included in the econjobmarket list of institutions.

This matrix says that each tier 1 university hired almost 2 and a half graduates every year from other tier 1 universities

These tended to be institutions who never placed an ad on econjobmarket, and who were never mentioned as a graduating institution
by any applicant who registered on econjobmarket.  Basically these were people who failed to get jobs on the international job market.
This is not completely accurate, but no definition of the scope of a market is ever going to be good.

The next computation just does the same thing for placements.  The third to last row

In [4]:
# tier placement rates 
ppr = zeros(size(adjacency)[1], size(adjacency)[2]);
for i in 1:size(adjacency)[1], j in 1:size(adjacency)[2]
    ppr[i,j] = 
    adjacency[i,j]/(classification_properties["num_years"]*classification_properties["institution_counts"][j])
end
ppr

12×5 Matrix{Float64}:
 2.56667    0.247312   0.042365   0.00580019  0.0
 2.0625     0.659946   0.0751862  0.0144413   0.000586854
 2.23125    0.967742   0.238594   0.0184659   0.00242077
 0.585417   0.381048   0.107542   0.0557528   0.00315434
 0.0166667  0.031586   0.0118715  0.00520833  0.0118838
 1.20208    0.402554   0.068203   0.00828598  0.00146714
 1.64792    0.308468   0.058892   0.00911458  0.00205399
 0.18125    0.106855   0.0332868  0.00769413  0.00212735
 0.0729167  0.0490591  0.0176909  0.00591856  0.00396127
 0.852083   0.501344   0.221136   0.0686553   0.0323504
 0.441667   0.120968   0.0228119  0.00414299  0.000660211
 0.4875     0.252016   0.0989292  0.0211884   0.00513498

When doing the probability calculations, the number of years over which data is collected is constant for all 
observations.  So whenever we do ratios it will just cancel out.  By recomputing rates to exclude it we can use the poisson
distribution directly for university level probability calculations.

In [5]:
hiring_rates = zeros(size(adjacency)[1], size(adjacency)[2]);
for i in 1:size(adjacency)[1], j in 1:size(adjacency)[2]
    hiring_rates[i,j] = 
    adjacency[i,j]/classification_properties["institution_counts"][i]
end
placement_rates = zeros(size(adjacency)[1], size(adjacency)[2]);
for i in 1:size(adjacency)[1], j in 1:size(adjacency)[2]
    placement_rates[i,j] = 
    adjacency[i,j]/classification_properties["institution_counts"][j]
end

The next task is to load the academic placements and the tier information

In [6]:
#academic_builder = load(files_path*"academic_builder.jld")["academic_builder"];
filtered_data = load(files_path*"filtered_data.jld")["filtered_data"];
sinks = load(files_path*"sinks.jld")["sinks"];
id_to_type_api = JSON.parsefile(files_path*"id_to_type_api.json");


In [7]:
function hiring_outcomes(filtered_data, id_to_type_api, num_types, ejm_id)
    cnt = zeros(Int64,num_types)
    #println("ejm_is is ",typeof(ejm_id), "\n")
    for placement in filtered_data
        if placement["to_institution_id"] ==  ejm_id
            #println(placement["from_institution_id"], " has type ", typeof(placement["from_institution_id"]))
            t = type_lookup(id_to_type_api,placement["from_institution_id"])
            #hack because from_institution_id = 1021 return nulls (Tech University of Brauschweig)
            # and this is the only missing id 1021 is type 5
            if t == 0 
                t = 5
            end
            #println(placement["from_institution_id"], " returns ", t, " which has type ",typeof(t))
            cnt[t] += 1
        end
    end
    return cnt
end

function placement_outcomes(filtered_data, sinks, id_to_type_api, num_types, ejm_id)
    cnt = zeros(Int64, num_types)
    for placement in filtered_data
        if placement["from_institution_id"] ==  string(ejm_id) || placement["from_institution_id"] ==  ejm_id
            #println(placement["to_name"])
            k = placement["to_institution_id"]
            t = type_lookup(id_to_type_api,string(k))
            #println(placement["from_institution_id"], " returns ", t, " which has type ",typeof(t))
            if t > 0
                cnt[t] += 1
            else
                p = placement["to_name"]
                #println("checking ", p)
                n = 1
                for sink in sinks
                    if p in sink
                        cnt[5+n] += 1 
                        break 
                    else
                        n += 1
                    end
                end
            end
        end
    end
    return cnt
end

function type_lookup(id_to_type_api, ejm_id)
    for x in id_to_type_api
        if x["institution_id"] == ejm_id
            return x["type"]
        end
    end
    return 0
end

function to_type_lookup(id_to_type_api, ejm_id)
    for x in id_to_type_api
        if x["institution_id"] == string(ejm_id)
            return x["type"]
        end
    end
    return 0
end

function name_lookup(id_to_type_api, ejm_id)
    for x in id_to_type_api
        if x["institution_id"] == string(ejm_id)
            return x["name"]
        end
    end
end

function institution_lookup(id_to_type_api, ejm_id)
    for x in id_to_type_api
        if x["institution_id"] == string(ejm_id)
            return x
        end
    end
end        

institution_lookup (generic function with 1 method)

In [8]:
all_data = load(files_path*"current_estimates.jld")["all_data"]
expected_offers = load(files_path*"expected_offers.jld")["expected_offers"]
offer_values = []
for j in 1:length(expected_offers)
    if j < classification_properties["unmatched_index"]-1
        push!(offer_values, expected_offers[j])
    elseif j == classification_properties["unmatched_index"]-1
        push!(offer_values, expected_offers[j])
        push!(offer_values, 0.0)
    else
        push!(offer_values,expected_offers[j-1])
    end
end
tier_values = []
for j in 1:5
    push!(tier_values, all_data[j])
end

In [9]:
h = ["Values"]
pretty_table(offer_values, header = h, row_labels = classification_properties["row_labels"])
#offer_values

┌───────────────────────────────────┬──────────┐
│                                   │   Values │
├───────────────────────────────────┼──────────┤
│                 TYPE 1 (20 insts) │ 0.578468 │
│                 TYPE 2 (62 insts) │ 0.516514 │
│                TYPE 3 (179 insts) │  0.49775 │
│                TYPE 4 (352 insts) │ 0.501864 │
│                TYPE 5 (568 insts) │ 0.499512 │
│         Public Sector (159 insts) │ 0.513628 │
│        Private Sector (255 insts) │ 0.498143 │
│              Postdocs (635 insts) │ 0.393563 │
│             Lecturers (443 insts) │ 0.501217 │
│               Unmatched (1 insts) │      0.0 │
│           Other Groups (39 insts) │ 0.501217 │
│ Teaching Universities (663 insts) │ 0.511279 │
└───────────────────────────────────┴──────────┘


In [10]:
path = files_path*"offer_values_table.tex"
open(path , "w") do f
        pretty_table(
            f,
            offer_values,
            header = h,
            row_labels = classification_properties["row_labels"],
            backend = Val(:latex)
            )
end

In [11]:
tier_values

5-element Vector{Any}:
 1.0
 0.6647461328472639
 0.10069973807987549
 0.08130366361549257
 0.0033203921074349955

In [12]:
function likelihood_ratios(filtered_data, sinks, id_to_type_api, hiring_rates, 
        placement_rates, offer_values, tier_values, institution_id)
    """
        compute likelihood ratios by calculating the mutinomial probability of hires and placements using
        the ml poission rates for each of the tiers.  Normally, the tier to which they are assigned should
        have ratio 1, but with small numbers of placements or hires this breaks down
    
        In cases with no recorded hires, rates are set arbirarily to 1 for the tier, zero for everything else
        Not sure yet what happens when placements or hires are small, eg 1 or 2
    """
    
    institution = institution_lookup(id_to_type_api, institution_id)
    r = zeros(size(hiring_rates)[2])
    p = zeros(size(hiring_rates)[2])
    q = zeros(size(hiring_rates)[2])
    euclid = zeros(size(hiring_rates)[2])
    hiring = zeros(size(hiring_rates)[2])
    placing = zeros(size(hiring_rates)[2])
    t = institution["type"]
    name = institution["name"]
    #println(t, " ", name)
    
    # start with hiring
    a = hiring_outcomes(filtered_data, id_to_type_api, 
        size(hiring_rates)[2], parse(Int64, institution["institution_id"]))
    placements = placement_outcomes(filtered_data, sinks, id_to_type_api, 
        size(hiring_rates)[1], institution["institution_id"])
    if iszero(a)
        for k in 1:size(hiring_rates)[2]
            if k == t
                r[k] = 1
                break
            end
        end
        return Dict("name" => name, "id" => institution["institution_id"],
            "tier" => t, "ratios" => r , "hires" => a, "placements" => placements,
            "hiring_value" => 0, "placement_value" => transpose(offer_values)*placements)
    end 
    for i in 1:size(hiring_rates)[2]
        prod = 1
        for j in 1:length(a)
            prod = pdf(Poisson(hiring_rates[i,j]), a[j])*prod
        end
        p[i] = prod
    end
    for j in 1:size(hiring_rates)[2]
        hiring[j]=p[j]/p[t]
    end
    
    for i in 1:size(hiring_rates)[2]
        prod = 1
        for j in 1:length(placements)
            prod = pdf(Poisson(placement_rates[j,i]), placements[j])*prod
        end
        q[i] = prod
    end
    for j in 1:size(hiring_rates)[2]
        placing[j]=q[j]/q[t]
    end
    for j in 1:size(hiring_rates)[2]
        r[j]=(p[j]*q[j])/(p[t]*q[t])
    end
    for i in 1:size(hiring_rates)[2]
        euclid[i] = norm(vcat(a,placements)-vcat(hiring_rates[i,:],placement_rates[:,i]))
    end
    value_of_hires = transpose(tier_values)*a
    value_of_placements = transpose(offer_values)*placements

    return  Dict("name" => name, "id" => institution["institution_id"],
        "tier" => t, "ratios" => r, "hiring_ratios" => hiring,
        "placement_ratios" => placing, "hires" => a, "placements" => placements, "euclidian" => euclid,
        "hiring_value" => value_of_hires, "placement_value" => value_of_placements)
end

likelihood_ratios (generic function with 1 method)

In [13]:
# run various tests
a = likelihood_ratios(filtered_data, sinks, id_to_type_api, hiring_rates, placement_rates,
    offer_values, tier_values, 361
)
#a = hiring_outcomes(filtered_data, id_to_type_api, size(placement_rates)[2], 1765)
#a = placement_outcomes(filtered_data, sinks, id_to_type_api, size(placement_rates)[1], 32)
#id_to_type_api[9]["institution_id"]
#type_lookup(id_to_type_api,"350")

Dict{String, Any} with 11 entries:
  "tier"             => 1
  "placement_ratios" => [1.0, 0.0955114, 1.22959e-91, 7.9561e-232, 0.0]
  "hires"            => [38, 23, 11, 2, 0]
  "placement_value"  => 83.722
  "name"             => "Cornell University"
  "euclidian"        => [59.8512, 42.057, 69.6828, 79.7036, 81.5303]
  "id"               => "361"
  "hiring_ratios"    => [1.0, 0.000138083, 7.35745e-20, 4.81678e-69, 1.3259e-17…
  "ratios"           => [1.0, 1.31885e-5, 9.04665e-111, 0.0, 0.0]
  "placements"       => [18, 29, 42, 8, 1, 14, 24, 4, 2, 23, 6, 16]
  "hiring_value"     => 54.5595

In [14]:


#save the calculations for all institutions
rep = []
for institution in id_to_type_api
    try
        s = likelihood_ratios(filtered_data, sinks, id_to_type_api, hiring_rates,
            placement_rates, offer_values, tier_values, institution["institution_id"])
        push!(rep, s)
    catch e
        rethrow(e)
    end
    
end
  

In [15]:
open(files_path*"likelihood_ratios.json", "w") do f
    write(f, JSON.json(rep))
end;

In [16]:
n = 0
m = 0
# set this to the tier you want to view
tier = 2
# set the next variable to zero to see all universities in the tier
# set it to 1 to see only those whose likelihood ratios don't support their tier assignment
anomalous = 0
for r in rep
    if r["tier"] == tier
        m += 1
        if maximum(r["ratios"]) > r["ratios"][r["tier"]]
            n +=1
            println(r["name"], " ", r["id"], " ", r["tier"])
            println("Anomalous ratio")
            for k in keys(r)
                println(k, " => ", r[k])
            end
            println("\n\n")
        else
            if anomalous == 1
                continue
            end
            println(r["name"], " ", r["id"], " ", r["tier"])
            for k in keys(r)
                println(k, " => ", r[k])
            end
            println("\n\n")
        end
    end
end
println("There were ", n, " anomalies")
println("The proportion ", n/m, " were anomalous") 

Arizona State University 370 2
tier => 2
placement_ratios => [3.288514032267142e-58, 1.0, 1.8762935721010107e-14, 1.5259362303491784e-65, 0.0]
hires => [17, 27, 3, 2, 0]
placement_value => 31.347736266560865
name => Arizona State University
euclidian => [101.21793319367866, 17.379614165843556, 32.367172322234126, 42.96773129896867, 45.08168539813745]
id => 370
hiring_ratios => [7.401589182271514e-11, 1.0, 1.4918091128944723e-8, 4.83356248526899e-37, 1.281558311410272e-105]
ratios => [2.4340229886976556e-68, 1.0, 2.7990718493256096e-22, 7.37570811792857e-102, 0.0]
placements => [2, 11, 22, 5, 0, 2, 5, 1, 0, 15, 2, 12]
hiring_value => 35.412852128346735



Bocconi University 1034 2
tier => 2
placement_ratios => [6.8695683308001e-38, 1.0, 5.684272931227145e-49, 4.118941651976738e-133, 0.0]
hires => [13, 34, 10, 8, 1]
placement_value => 57.487877113917676
name => Bocconi University
euclidian => [93.35400366347444, 24.554469171528137, 46.85276224696596, 56.77396191179712, 59.113812813748474